The intent of this notebook is two-fold:

1) Try to see which networkx graph generators most closely mimic the data of Cornell university student network given here https://osf.io/t7n9f/

2) Test the time dependence of Max-Cut on the size of a graph generated by the most accurate generator determined in part 1)

In [64]:
from collections import defaultdict
import numpy as np
from itertools import combinations
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
from matplotlib import pyplot as plt
import networkx as nx
import networkx.generators.community as community
from networkx.generators.random_graphs import connected_watts_strogatz_graph as small_world
from networkx.generators.random_graphs import powerlaw_cluster_graph as cluster
import ndlib
import future.utils
import warnings
import time
from random import randint
from random import sample
warnings.simplefilter('ignore')
%matplotlib inline

In [2]:
# Create QUBO dict for D-Wave to optimize
def maxcut_qubo(G):
    # The Hamiltonian is H = sum(Pauli_Z(i) x Pauli_Z(j) - I)/2
    # The eigenvalue is the negation of the edges in cut solution
    # The QUBO should be -xi-xj+2xixj, we label it as Q
    Q = defaultdict(int)
    for u, v in G.edges:
        Q[(u,u)]+= -1
        Q[(v,v)]+= -1
        Q[(u,v)]+= 2
    return Q

# Invoke D-Wave annealer
def solve(Q, chainstrength=8, numruns=100):
    # Run the QUBO on the solver from your config file
    sampler = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))
    response = sampler.sample_qubo(Q, num_reads=numruns)
    energies = iter(response.data())
    return response

# Testing different graph generators

https://osf.io/t7n9f/?show=view gives the 
##### target stats for Cornell University model:

Number of unique nodes  3,800

Number of unique edges 287,327 

Network density .040

Clustering coefficient .480

Average geodesic distance 2.233

Network diameter (largest observed distance) 65

Average k-step reach centrality: k=1::   0.040,  k=2::   0.725,  k=3::   0.994,  k=4::   0.99

In [5]:
def random_avg_distance(G, samples):
    distance_sum = 0
    for i in range(samples):
        pair = sample(list(G.nodes()), 2)
        #print(pair)
        distance = len(nx.shortest_path(G, pair[0], pair[1]))
        #print(distance)
        distance_sum += distance
    return distance_sum/samples

def random_avg_clustering_coeff(G, samples):
    sampled_nodes = sample(list(G.nodes()), samples)
    clustering_dict = nx.clustering(G, sampled_nodes)
    return np.mean(np.array(list(clustering_dict.values())))

## small world

##### connected_watts_strogatz_graph(n, k, p, tries=100, seed=None)

Parameters:

n: number of nodes 

k: how many nearest neighbors in ring to connect each node to 

p: prob to rewire edge

In [100]:
big_small_world = small_world(3800, 152, .15)
print('density: ', nx.density(big_small_world))
print('cluster coeff:', random_avg_clustering_coeff(big_small_world, 1000))
print('avg distance:', random_avg_distance(big_small_world, 1000))

matching density and clustering coefficient, avg geodesic distance a bit too high

#### newman_watts_strogatz_graph(n, k, p, seed=None)

Returns a Newman–Watts–Strogatz small-world graph.

Parameters:

n (int) – The number of nodes.

k (int) – Each node is joined with its k nearest neighbors in a ring topology.

p (float) – The probability of adding a new edge for each edge.

seed (integer, random_state, or None (default)) – Indicator of random number generation state. See Randomness.

In [109]:
big_nw_small_world = nx.generators.random_graphs.newman_watts_strogatz_graph(3800, 150, .5)
print('density: ', nx.density(big_nw_small_world))
print('cluster coeff:', random_avg_clustering_coeff(big_nw_small_world, 1000))
print('avg distance:', random_avg_distance(big_nw_small_world, 1000))

density:  0.05917637605464042
cluster coeff: 0.35473747639591363
avg distance: 2.938


Unable to improve on avg distance 

## power law cluster graph

#### powerlaw_cluster_graph(n, m, p, seed=None) 

Parameters:

n (int) – the number of nodes

m (int) – the number of random edges to add for each new node

p (float,) – Probability of adding a triangle after adding a random edge

In [26]:
big_cluster = cluster(3800, 70, 1)
print('density: ', nx.density(big_cluster))
print('cluster coeff:', random_avg_clustering_coeff(big_cluster, 1000))
print('avg distance:', random_avg_distance(big_cluster, 1000))

bound on clustering coefficient, cant get it above around .25, so does not match data

## community graphs

#### planted_partition_graph(l, k, p_in, p_out, seed=None, directed=False)[source]
Returns the planted l-partition graph.

This model partitions a graph with n=l*k vertices in l groups with k vertices each. Vertices of the same group are linked with a probability p_in, and vertices of different groups are linked with probability p_out.

Parameters:

l (int) – Number of groups

k (int) – Number of vertices in each group

p_in (float) – probability of connecting vertices within a group

p_out (float) – probability of connected vertices between groups

In [59]:
ppg = community.planted_partition_graph(10, int(3800/10), 1, .05)
print('density: ', nx.density(ppg))
print('cluster coeff:', random_avg_clustering_coeff(ppg, 20))
print('avg distance:', random_avg_distance(ppg, 1000))

Cant seem to get density low and clustering high enough simulataneously with this model

#### gaussian_random_partition_graph(n, s, v, p_in, p_out, directed=False, seed=None)[source]
Generate a Gaussian random partition graph.

A Gaussian random partition graph is created by creating k partitions each with a size drawn from a normal distribution with mean s and variance s/v. Nodes are connected within clusters with probability p_in and between clusters with probability p_out[1]

Parameters:

n (int) – Number of nodes in the graph

s (float) – Mean cluster size

v (float) – Shape parameter. The variance of cluster size distribution is s/v.

p_in (float) – Probabilty of intra cluster connection.

p_out (float) – Probability of inter cluster connection.

In [89]:
grpg = community.gaussian_random_partition_graph(3800, 1000, 100, .6, .1)
print('density: ', nx.density(grpg))
print('cluster coeff:', random_avg_clustering_coeff(grpg, 20))
print('avg distance:', random_avg_distance(grpg, 1000))

Wasn't able to get the exact stats

So it looks like the Watts-Strogatz graphs get closest to the Cornell class enrollment network. This is also suggested in the original paper

# How does D-Wave max-cut time depend on watts-strogatz graph size

In [61]:
densities, cluster_coeffs, avg_distances = [], [], []
a = small_world(10, 4, .05)
densities.append(nx.density(a))
cluster_coeffs.append(random_avg_clustering_coeff(a, 10))
avg_distances.append(random_avg_distance(a, 10))
b = small_world(30, 4, 0)
densities.append(nx.density(b))
cluster_coeffs.append(random_avg_clustering_coeff(b, 10))
avg_distances.append(random_avg_distance(b, 10))
c = small_world(50, 4, .05)
densities.append(nx.density(c))
cluster_coeffs.append(random_avg_clustering_coeff(c, 10))
avg_distances.append(random_avg_distance(c, 10))
d = small_world(100, 6, .1)
densities.append(nx.density(d))
cluster_coeffs.append(random_avg_clustering_coeff(d, 10))
avg_distances.append(random_avg_distance(d, 10))
e = small_world(150, 7, .1)
densities.append(nx.density(e))
cluster_coeffs.append(random_avg_clustering_coeff(e, 10))
avg_distances.append(random_avg_distance(e, 10))
f = small_world(180, 9, .11)
densities.append(nx.density(f))
cluster_coeffs.append(random_avg_clustering_coeff(f, 10))
avg_distances.append(random_avg_distance(f, 10))
print(densities) 
print(cluster_coeffs) 
print(avg_distances)

[0.4444444444444444, 0.13793103448275862, 0.08163265306122448, 0.06060606060606061, 0.040268456375838924, 0.0446927374301676]
[0.43999999999999995, 0.5, 0.43, 0.4709523809523809, 0.4766666666666667, 0.45]
[3.1, 4.6, 5.2, 5.2, 5.9, 5.2]


In [62]:
small_world_graphs = [a, b, c, d, e, f]
small_world_times = []
small_world_start_time = time.time()
small_world_previous_time = small_world_start_time
for graph, size in zip(small_world_graphs, [10, 30, 50, 100, 150, 180]):
    result = solve(maxcut_qubo(graph))
    small_world_current_time = time.time()
    small_world_times.append(small_world_current_time - small_world_previous_time)
    small_world_previous_time = small_world_current_time
    print(f'completed size {size}')
print(small_world_times)

completed size 10
completed size 30
completed size 50
completed size 100
completed size 150
completed size 180
[2.3966965675354004, 3.1521782875061035, 2.5011050701141357, 2.6607565879821777, 8.48445463180542, 7.3369739055633545]


Conclusion: We were able to embed small world (Watts-Strogatz) graphs of size up to 180 on D-Wave hardware in less than 10 seconds. We have not successfully been able to embed a graph of 200 nodes

Contributors:
    James Sud